In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import casadi as cd
import numpy as np

def step(x,u,dt):
    return x+u*dt


def dynamics_step( base_term, state_dot, dt ):
    next_state = base_term + state_dot * dt
    return next_state

# assume a single control input
# assume this is true dynamics
def dynamics_xdot_noisy(state, action):
    xdot = cd.vcat(
        [
            state[0]*state[0], state[1]*state[1]
        ]
    )
    # cov = cd.MX.zeros((2,2))
    cov = np.zeros((2,2))
    return xdot, cov

def get_mean( sigma_points, weights ):
    weighted_points = sigma_points * cd.repmat(weights,sigma_points.shape[0], 1 )
    mu = cd.sum2(weighted_points)
    return mu

def get_mean_cov(sigma_points, weights):
    
    # mean
    weighted_points = sigma_points * cd.repmat(weights,sigma_points.shape[0], 1 )
    mu = cd.sum2(weighted_points)
    
    # covariance
    centered_points = sigma_points - mu
    cov = cd.diag( cd.sum2( centered_points*centered_points * cd.repmat(weights,sigma_points.shape[0], 1 ) ) )
    return mu, cov

def get_ut_cov_root_diagonal(cov):
    offset = 0.000  # TODO: make sure not zero here
    # root_term = cd.diag( cd.diag(cov) + offset )
    root0 = cd.sqrt((offset+cov[0,0]))
    root1 = cd.sqrt((offset+cov[1,1]))    
    root_term = cd.diag( cd.vcat([root0, root1])[:,0] )
    print(f"root_term: {root_term}")
    return root_term

def get_mean_cov_skew_kurt( sigma_points, weights ):
    # mean
    weighted_points = sigma_points * cd.repmat(weights,sigma_points.shape[0], 1 )
    mu = cd.sum2(weighted_points)
    centered_points = sigma_points - mu
    cov = cd.diag( cd.sum2( centered_points*centered_points * cd.repmat(weights,sigma_points.shape[0], 1 ) ) )
    
    skewness = cd.sum2( centered_points**3 * cd.repmat(weights,sigma_points.shape[0], 1 ) ) 
   
    kurt = cd.sum2( centered_points**4 * cd.repmat(weights,sigma_points.shape[0], 1 ) )
    
    return mu, cov, skewness, kurt

def get_mean_cov_skew_kurt_for_generation( sigma_points, weights ):
    # mean
    weighted_points = sigma_points * cd.repmat(weights,sigma_points.shape[0], 1 )
    mu = cd.sum2(weighted_points)
    centered_points = sigma_points - mu
    cov = cd.diag( cd.sum2( centered_points*centered_points * cd.repmat(weights,sigma_points.shape[0], 1 ) ) )
    
    skewness_temp = cd.sum2( centered_points**3 * cd.repmat(weights,sigma_points.shape[0], 1 ) ) 
    skewness = cd.vcat(
        [
            skewness_temp[0] / cov[0,0]**(3/2),
            skewness_temp[1] / cov[1,1]**(3/2),
            skewness_temp[2] / cov[2,2]**(3/2),
            skewness_temp[3] / cov[3,3]**(3/2)
        ]
    )
   
    kurt_temp = cd.sum2( centered_points**4 * cd.repmat(weights,sigma_points.shape[0], 1 ) )
    kurt = cd.vcat(
        [
            kurt_temp[0]/cov[0,0]**(4/2),
            kurt_temp[1]/cov[1,1]**(4/2),
            kurt_temp[2]/cov[2,2]**(4/2),
            kurt_temp[3]/cov[3,3]**(4/2)
        ]
    )
    
    return mu, cov, skewness, kurt

def generate_sigma_points_gaussian( mu, cov_root, base_term, factor ):
    n = mu.shape[0]     
    N = 2*n + 1 # total points

    alpha = 1.0
    beta = 0.0#2.0#2.0 # optimal for gaussian
    k = 1.0
    Lambda = alpha**2 * ( n+k ) - n
    
    points0 = base_term + mu
    weights0 = [1.0*Lambda/(n+Lambda)] 
    
    points0 = base_term + mu * factor
    points1 = base_term + (mu + cd.sqrt(n+Lambda) * cov_root) * factor
    points2 = base_term + (mu - cd.sqrt(n+Lambda) * cov_root) * factor
    new_points = cd.hcat([
        points0, points1, points2
    ])
    
    weights0 = [1.0*Lambda/(n+Lambda)]
    weights1 = 1.0/(n+Lambda)/2.0 * np.ones((1,n))#cd.MX(np.ones((1,n)))
    weights2 = 1.0/(n+Lambda)/2.0 * np.ones((1,n))#cd.MX(np.ones((1,n)))
    new_weights = cd.hcat([
        weights0, weights1, weights2
    ])
    # new_weights_cov = weights
    

    return new_points, new_weights

def generate_sigma_points_gaussian_GenUT( mu, cov_root, skewness, kurt, base_term, factor ):
    n = mu.shape[0]     
    N = 2*n + 1 # total points
    
    u = 0.5 * ( - skewness + cd.sqrt( 4 * kurt - 3 * ( skewness )**2 ) )
    v = u + skewness

    # Weights
    w2 = (1.0 / v) / (u+v)
    w1 = (w2 * v) / u
    w0 = 1 - cd.sum()
    w0 = 1 - cd.sum1(w1) - cd.sum1(w2)
    
    # Points
    U = cd.diag(u)
    V = cd.diag(v)
    points0 = base_term + mu * factor
    points1 = base_term + (mu - cd.mtimes(cov_root, U)) * factor
    points2 = base_term + (mu + cd.mtimes(cov_root, V)) * factor
      
    # New sigma points
    new_points = cd.hcat( [points0, points1, points2] )
    new_weights = cd.hcat( [w0, w1, w2] )

    return new_points, new_weights

def sigma_point_expand(sigma_points, weights, control, dt):
   
    n, N = sigma_points.shape   

    mu, cov = dynamics_xdot_noisy(sigma_points[:,0], control)
    root_term = get_ut_cov_root_diagonal(cov) 
    new_points, temp_weights = generate_sigma_points_gaussian( mu, root_term, sigma_points[:,0], dt )
    new_weights = temp_weights * weights[0,0]
        
    for i in range(1,N):
        mu, cov = dynamics_xdot_noisy(sigma_points[:,i], control)
        root_term = get_ut_cov_root_diagonal(cov)           
        temp_points, temp_weights = generate_sigma_points_gaussian( mu, root_term, sigma_points[:,i], dt )
        new_points = cd.hcat([ new_points, temp_points ])
        new_weights = cd.hcat([ new_weights, temp_weights * weights[0,i]  ])

    return new_points, new_weights

def sigma_point_compress( sigma_points, weights ):
    mu, cov = get_mean_cov( sigma_points, weights )
    cov_root_term = get_ut_cov_root_diagonal( cov )  
    base_term =  np.zeros(mu.shape)#cd.MX.zeros(mu.shape)# 0*mu  
    return generate_sigma_points_gaussian( mu, cov_root_term, base_term, 1.0 )

def sigma_point_compress_GenUT( sigma_points, weights ):
    mu, cov, skewness, kurt = get_mean_cov_skew_kurt_for_generation( sigma_points, weights )
    cov_root_term = get_ut_cov_root_diagonal( cov )  
    base_term =  np.zeros(mu.shape)#cd.MX.zeros(mu.shape)
    return generate_sigma_points_gaussian_GenUT( mu, cov_root_term, skewness, kurt, base_term, 1.0 )

def foresee_propagate_GenUT( sigma_points, weights, action, dt ):
    
    expanded_sigma_points, expanded_weights = sigma_point_expand( sigma_points, weights, action, dt )
    compressed_sigma_points, compressed_weights = sigma_point_compress_GenUT(expanded_sigma_points, expanded_weights)
    return compressed_sigma_points, compressed_weights

def foresee_propagate( sigma_points, weights, action, dt ):

    #Expansion Layer
    expanded_sigma_points, expanded_weights = sigma_point_expand( sigma_points, weights, action, dt )
    compressed_sigma_points, compressed_weights = sigma_point_compress(expanded_sigma_points, expanded_weights)
    return compressed_sigma_points, compressed_weights

In [2]:
points, weights = generate_sigma_points_gaussian(np.zeros((2,1)), 2*np.eye(2), np.zeros((2,1)), 1.0)
print(points)
print(weights)


[[0, 3.4641, 0, -3.4641, 0], 
 [0, 0, 3.4641, 0, -3.4641]]
[[0.333333, 0.166667, 0.166667, 0.166667, 0.166667]]


In [3]:
get_mean_cov(points, weights)

(DM([0, 0]),
 DM(
 [[4, 00], 
  [00, 4]]))

In [4]:
N = 1
points = np.append( np.ones((1,N)), 2*np.ones((1,N)), axis=0 )
# print(points)
weights = np.ones((1,N))/N
points, weights = sigma_point_expand( points, weights,np.zeros((2,1)), 1.0 )
# print(points)
# print(weights)
points2, weights2 = sigma_point_expand( points, weights,np.zeros((2,1)), 1.0 )
print(points2)
print(weights2)
np.sum(weights2)

root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]

[[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], 
 [42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42]]
[[0.111111, 0.0555556, 0.0555556, 0.0555556, 0.0555556, 0.0555556, 0.0277778, 0.0277778, 0.0277778, 0.0277778, 0.0555556, 0.0277778, 0.0277778, 0.0277778, 0.0277778, 0.0555556, 0.0277778, 0.0277778, 0.0277778, 0.0277778, 0.0555556, 0.0277778, 0.0277778, 0.0277778, 0.0277778]]


1.0

In [5]:
N = 5
sigma_points, weights = generate_sigma_points_gaussian( np.ones((2,1)), np.zeros((2,2)), np.zeros((2,1)), 1.0 )
print(f"{sigma_points}, {weights}")
points, weights = sigma_point_expand( points, weights,np.zeros((2,1)), 1.0 )
foresee_propagate( sigma_points, weights, np.zeros((2,1)), 1.0 )


[[1, 1, 1, 1, 1], 
 [1, 1, 1, 1, 1]], [[0.333333, 0.166667, 0.166667, 0.166667, 0.166667]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0.7698, 00], 
 [00, 0.7698]]


(DM(
 [[0.666667, 2, 0.666667, -0.666667, 0.666667], 
  [0.666667, 0.666667, 2, 0.666667, -0.666667]]),
 DM([[0.333333, 0.166667, 0.166667, 0.166667, 0.166667]]))

In [6]:
def test():
    N = 5
    sigma_points, weights = generate_sigma_points_gaussian( np.ones((2,1)), np.zeros((2,2)), np.zeros((2,1)), 1.0 )
    foresee_propagate( sigma_points, weights, np.zeros((2,1)), 1.0 )
    return

In [7]:
%timeit test()

root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[4.44089e-16, 00], 
 [00, 4.44089e-16]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[4.44089e-16, 00], 
 [00, 4.44089e-16]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[4.44089e-16, 00], 
 [00, 4.44089e-16]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[4.44089e-16, 00], 
 [00, 4.44089e-16]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 0]]
root_term: 
[[0, 00], 
 [00, 

In [1]:
0.12997911+ 0.13643052

0.26640963

In [2]:
[0.12997911, 0.13643052]

[0.12997911, 0.13643052]